# Neutryx Risk Analysis Demo

This notebook demonstrates risk analysis capabilities:
- Portfolio Greeks aggregation
- VaR calculation
- Stress testing scenarios
- Exposure profiles

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

## 1. Sample Portfolio

Create a sample derivatives portfolio.

In [ ]:
# Load sample trade data
equity_trades = pd.read_csv('../data/input/trades/equity_book.csv')
rates_trades = pd.read_csv('../data/input/trades/rates_book.csv')
fx_trades = pd.read_csv('../data/input/trades/fx_book.csv')

print(f"Equity trades: {len(equity_trades)}")
print(f"Rates trades: {len(rates_trades)}")
print(f"FX trades: {len(fx_trades)}")

In [ ]:
# Display sample trades
print("\nEquity Book Sample:")
print(equity_trades[['trade_id', 'instrument_type', 'notional', 'currency']].head())

## 2. Portfolio Greeks Aggregation

In [ ]:
# Simulated Greeks for the portfolio
np.random.seed(42)

n_trades = len(equity_trades) + len(rates_trades) + len(fx_trades)
portfolio_greeks = pd.DataFrame({
    'trade_id': [f'T{i:03d}' for i in range(n_trades)],
    'notional': np.random.uniform(1e6, 1e8, n_trades),
    'pv': np.random.uniform(-1e6, 1e6, n_trades),
    'delta': np.random.uniform(-1, 1, n_trades),
    'gamma': np.random.uniform(0, 0.1, n_trades),
    'vega': np.random.uniform(-10000, 10000, n_trades),
    'theta': np.random.uniform(-5000, 0, n_trades)
})

# Aggregate
total_pv = portfolio_greeks['pv'].sum()
total_delta = portfolio_greeks['delta'].sum()
total_gamma = portfolio_greeks['gamma'].sum()
total_vega = portfolio_greeks['vega'].sum()
total_theta = portfolio_greeks['theta'].sum()

print("Portfolio Greeks Summary:")
print(f"  Total PV:    ${total_pv:,.0f}")
print(f"  Total Delta: {total_delta:,.2f}")
print(f"  Total Gamma: {total_gamma:,.4f}")
print(f"  Total Vega:  ${total_vega:,.0f}")
print(f"  Total Theta: ${total_theta:,.0f}/day")

## 3. Value at Risk (VaR)

In [ ]:
# Historical VaR simulation
n_scenarios = 10000
holding_period = 10  # days
confidence = 0.99

# Simulate returns
daily_vol = 0.02  # 2% daily volatility
returns = np.random.normal(0, daily_vol * np.sqrt(holding_period), n_scenarios)

# Portfolio P&L
portfolio_value = abs(total_pv) + 1e7  # Base portfolio value
pnl = portfolio_value * returns

# Calculate VaR
var_99 = np.percentile(pnl, (1 - confidence) * 100)
cvar_99 = pnl[pnl <= var_99].mean()

print(f"10-Day VaR (99%): ${-var_99:,.0f}")
print(f"10-Day CVaR (99%): ${-cvar_99:,.0f}")

# Plot distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(pnl, bins=100, density=True, alpha=0.7, color='blue')
plt.axvline(x=var_99, color='red', linestyle='--', linewidth=2, label=f'VaR 99%: ${-var_99:,.0f}')
plt.xlabel('P&L ($)')
plt.ylabel('Density')
plt.title('Portfolio P&L Distribution')
plt.legend()

plt.subplot(1, 2, 2)
sorted_pnl = np.sort(pnl)
plt.plot(np.arange(len(sorted_pnl))/len(sorted_pnl) * 100, sorted_pnl, 'b-')
plt.axhline(y=var_99, color='red', linestyle='--', label='VaR 99%')
plt.xlabel('Percentile')
plt.ylabel('P&L ($)')
plt.title('P&L Percentile Chart')
plt.legend()

plt.tight_layout()
plt.show()

## 4. Stress Testing

In [ ]:
# Define stress scenarios
scenarios = {
    'IR +100bp': {'ir_shock': 0.01, 'fx_shock': 0, 'eq_shock': -0.05},
    'IR -100bp': {'ir_shock': -0.01, 'fx_shock': 0, 'eq_shock': 0.02},
    'Equity -20%': {'ir_shock': 0, 'fx_shock': 0.05, 'eq_shock': -0.20},
    'FX Crisis': {'ir_shock': 0.005, 'fx_shock': 0.15, 'eq_shock': -0.10},
    'Vol +25%': {'ir_shock': 0, 'fx_shock': 0, 'eq_shock': -0.08},
    'Credit Event': {'ir_shock': 0.02, 'fx_shock': 0.08, 'eq_shock': -0.15}
}

# Calculate P&L under each scenario
base_pv = portfolio_value
stress_results = []

for scenario_name, shocks in scenarios.items():
    # Simplified P&L impact
    ir_impact = shocks['ir_shock'] * total_delta * 1e6
    fx_impact = shocks['fx_shock'] * base_pv * 0.1
    eq_impact = shocks['eq_shock'] * base_pv * 0.3
    total_impact = ir_impact + fx_impact + eq_impact
    
    stress_results.append({
        'Scenario': scenario_name,
        'P&L Impact': total_impact,
        '% Impact': total_impact / base_pv * 100
    })

stress_df = pd.DataFrame(stress_results)
print("Stress Test Results:")
print(stress_df.to_string(index=False))

# Plot
plt.figure(figsize=(10, 6))
colors = ['red' if x < 0 else 'green' for x in stress_df['P&L Impact']]
plt.barh(stress_df['Scenario'], stress_df['P&L Impact'] / 1e6, color=colors, alpha=0.7)
plt.xlabel('P&L Impact ($ millions)')
plt.title('Stress Test Results')
plt.axvline(x=0, color='black', linewidth=0.5)
plt.grid(True, axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Exposure Profile

In [ ]:
# Generate exposure profile over time
time_grid = np.linspace(0, 10, 41)  # 10 years, quarterly
n_paths = 1000

# Simulate MtM paths (simplified Brownian motion)
dt = time_grid[1] - time_grid[0]
vol = 0.1
drift = 0.0

paths = np.zeros((n_paths, len(time_grid)))
paths[:, 0] = base_pv

for i in range(1, len(time_grid)):
    dW = np.random.normal(0, np.sqrt(dt), n_paths)
    paths[:, i] = paths[:, i-1] * np.exp((drift - 0.5*vol**2)*dt + vol*dW)

# Calculate exposure metrics
positive_exposure = np.maximum(paths, 0)
ee = positive_exposure.mean(axis=0)  # Expected Exposure
pfe_95 = np.percentile(positive_exposure, 95, axis=0)  # PFE 95%
pfe_99 = np.percentile(positive_exposure, 99, axis=0)  # PFE 99%
epe = ee.mean()  # Expected Positive Exposure

print(f"EPE: ${epe:,.0f}")
print(f"Peak EE: ${ee.max():,.0f} at T={time_grid[ee.argmax()]:.1f}Y")
print(f"Peak PFE 95%: ${pfe_95.max():,.0f}")

# Plot exposure profile
plt.figure(figsize=(12, 6))
plt.fill_between(time_grid, 0, pfe_99, alpha=0.2, color='red', label='PFE 99%')
plt.fill_between(time_grid, 0, pfe_95, alpha=0.3, color='orange', label='PFE 95%')
plt.fill_between(time_grid, 0, ee, alpha=0.4, color='blue', label='EE')
plt.axhline(y=epe, color='green', linestyle='--', label=f'EPE: ${epe/1e6:.1f}M')
plt.xlabel('Time (Years)')
plt.ylabel('Exposure ($)')
plt.title('Counterparty Credit Exposure Profile')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
1. Portfolio Greeks aggregation
2. Historical VaR and CVaR calculation
3. Scenario-based stress testing
4. Counterparty credit exposure profiles (EE, EPE, PFE)

Neutryx's `pricer_risk` crate provides production-grade implementations of these risk analytics.